In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
sdir = str(input('Enter the working directory: '))
fits_name = str(input('Enter the FITS file name: '))

sdir_spectra = sdir + 'Spectra/'

if not os.path.exists(sdir_spectra):
    os.makedirs(sdir_spectra)  

Enter the working directory: /Users/thepoetoftwilight/Documents/SOFIA_FIFI_Cycle-8/Data/NGC_5253_sw_1/
Enter the FITS file name: NGC_5253_sw_1.fits


In [3]:
hdulist = fits.open(sdir + fits_name)

hdulist.info()

Filename: /Users/thepoetoftwilight/Documents/SOFIA_FIFI_Cycle-8/Data/NGC_5253_sw_1/NGC_5253_sw_1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     311   ()      
  1  FLUX          1 ImageHDU        28   (33, 35, 56)   float64   
  2  ERROR         1 ImageHDU        28   (33, 35, 56)   float64   
  3  UNCORRECTED_FLUX    1 ImageHDU        28   (33, 35, 56)   float64   
  4  UNCORRECTED_ERROR    1 ImageHDU        28   (33, 35, 56)   float64   
  5  WAVELENGTH    1 ImageHDU         7   (56,)   float64   
  6  X             1 ImageHDU         7   (33,)   float64   
  7  Y             1 ImageHDU         7   (35,)   float64   
  8  TRANSMISSION    1 ImageHDU         7   (56,)   float64   
  9  RESPONSE      1 ImageHDU         7   (56,)   float32   
 10  EXPOSURE_MAP    1 ImageHDU        28   (33, 35, 56)   int16   
 11  UNSMOOTHED_TRANSMISSION    1 ImageHDU         8   (1540, 2)   float32   


In [4]:
wavelengths = hdulist[5].data

x_coords = hdulist[6].data
x_min = x_coords[0]
x_max = x_coords[len(x_coords)-1]

y_coords = hdulist[7].data
y_min = y_coords[0]
y_max = y_coords[len(y_coords)-1]

In [5]:
# wavelength is the first dimension
# y is the second dimension
# x is the third dimension

fluxmaps = hdulist[1].data

fluxmaps_min = np.nanmin(fluxmaps.flatten())
fluxmaps_max = np.nanmax(fluxmaps.flatten())

print(fluxmaps.shape)

(56, 35, 33)


In [6]:
def sum_spaxel(fluxmap, k_x, k_y):
    
    n_x = len(fluxmap[0])
    n_y = len(fluxmap)
    
    id_x = np.arange(np.floor((n_x-k_x)/2), np.floor((n_x-k_x)/2) + k_x, 1)
    id_x = id_x.astype(int)
    
    id_y = np.arange(np.floor((n_y-k_y)/2), np.floor((n_y-k_y)/2) + k_y, 1)
    id_y = id_y.astype(int)
    
    central_fluxmap = []
    
    for j in id_y:
        
        y_snip = []
        
        for i in id_x:
            
            y_snip.append(fluxmap[j][i])
            
        central_fluxmap.append(y_snip)
            
    central_fluxmap = np.array(central_fluxmap)
    
    central_flux = np.sum(central_fluxmap)
    
    return central_flux

In [7]:
central_fluxes = []

n_x = 3
n_y = 3

for fluxmap in fluxmaps:
    
    central_flux = sum_spaxel(fluxmap, n_x, n_y)
    central_fluxes.append(central_flux)
    
central_fluxes = np.array(central_fluxes)

In [8]:
fig, ax = plt.subplots(figsize = (18, 13))

ax.plot(wavelengths, central_fluxes, label = '3 x 3', lw = 5, color = 'orange')

ax.set_xlabel(r'$\lambda$ (nm)', fontsize = 38, labelpad = 5)
ax.set_ylabel('Flux', fontsize = 38, labelpad = 10)

ax.tick_params(labelsize = 38, pad = 10)
ax.yaxis.offsetText.set_fontsize(36)

ax.legend(prop={'size': 38}, loc = 'upper right')

plt.savefig(sdir_spectra + '{0}x{1}-spectrum.png'.format(str(n_x), str(n_y)))
plt.close()